Цей код створює 2 окремі датафрейми:
- df_duration_ouliers - містить аутлаєри по часу поїздки > 24 годин.
- df - містить сирі дані очищені від аутлаєрів > 24 годин.

В обох датафреймах міститься ще 2 додаткові колонки: started_at_dt і ended_at_dt, що містять час початку і кінця поїздки в типі datetime в таймзомі міста Осло.

In [40]:
import pandas as pd
import datetime
import time
import helpful_functions
import pytz

In [56]:
city_name = 'oslo'        # можливі варіанти: olso, bergen, trondheim

start_month = 4           # 1 - 12
start_year = 2019         # 2019 - 2022

end_month = 6             # 1 - 12
end_year = 2022           # 2019 - 2022

data_folder = './data'

In [57]:
dates_str_list = helpful_functions.generate_months_in_period(
    start_month=start_month, 
    start_year=start_year, 
    end_month=end_month, 
    end_year=end_year, 
    sep="_")

In [58]:
start = time.time()
df = pd.read_csv(f"{data_folder}/{city_name}/{city_name}_{dates_str_list[0]}.csv")
for date in dates_str_list[1:]:
    try:
        df = pd.concat([df, pd.read_csv(f"{data_folder}/{city_name}/{city_name}_{date}.csv")])
    except:
        pass
end = time.time()
print("Зчитування: ", end - start, "с. \n")
print("Розмір початкової таблиці: ", df.shape)

Зчитування:  48.43748593330383 с. 

Розмір початкової таблиці:  (5833647, 13)


In [ ]:
strptime_pattern = "%Y-%m-%d %H:%M:%S%z"
timezone = pytz.timezone('Europe/Oslo')

start = time.time()
df['started_at_dt'] = df['started_at'].apply(lambda x: datetime.datetime.strptime(x[:-6].split('.')[0] + x[-6:], strptime_pattern).astimezone(timezone))
df['ended_at_dt'] = df['ended_at'].apply(lambda x: datetime.datetime.strptime(x[:-6].split('.')[0] + x[-6:], strptime_pattern).astimezone(timezone))
end = time.time()
print("Трансформування: ", end - start, "с.")
df.head()

In [ ]:
df_start_stations = df[['start_station_id',
                        'start_station_name',
                           'start_station_description', 
                           'start_station_latitude', 
                           'start_station_longitude']]

In [ ]:
df_start_stations = df_start_stations.drop_duplicates(subset=['start_station_id'],
                                                     keep='last')

In [ ]:
df_start_stations.shape

In [ ]:
df_start_stations = df_start_stations.rename(columns={'start_station_id': 'station_id',
                                                     'start_station_name': 'station_name',
                                                     'start_station_description': 'station_description',
                                                     'start_station_latitude': 'station_latitude',
                                                     'start_station_longitude': 'station_longitude'})

In [ ]:
df_start_stations.to_csv(f'./data/{city_name}_stations.csv', index=False)

In [ ]:
df = df[['started_at_dt', 'ended_at_dt', 'duration', 'start_station_id', 'end_station_id']]

In [ ]:
df.shape

Створюємо 2 датафрейми: 
- df_duration_ouliers - містить аутлаєри по часу поїздки > 24 годин. Потрібний для подальшого аналізу цих аутлаєрів.
- df - містить сирі дані очищені від аутлаєрів > 24 годин. Для подальшого аналізу.

In [ ]:
df_duration_outliers = df[df['duration'] > 24 * 60 * 60]
df = df[df['duration'] <= 24 * 60 * 60]

In [ ]:
start = time.time()
df.to_csv(f'./data/{city_name}_clear_rides_data.csv', index=False)
end = time.time()
end - start

In [ ]:
start = time.time()
df_duration_outliers.to_csv(f'./data/{city_name}_duration_outliers-stolen_bikes.csv', index=False)
end = time.time()
end - start